In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import BlockRNNModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 72)
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    layer_size = trial.suggest_int("layer_size", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 64, 256)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    # Try different loss functions
    loss_functions = {
        'mse': torch.nn.MSELoss(),
        'mae': torch.nn.L1Loss(),
        'huber': torch.nn.HuberLoss(delta=1.0),
        'smoothl1': torch.nn.SmoothL1Loss()
    }
    loss_name = trial.suggest_categorical("loss_fn", list(loss_functions.keys()))

    # Try different optimizers
    optimizer_names = ['Adam', 'AdamW', 'RMSprop', 'SGD']
    optimizer_name = trial.suggest_categorical("optimizer", optimizer_names)
    optimizer_dict = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW,
        'RMSprop': torch.optim.RMSprop,
        'SGD': torch.optim.SGD
    }

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }
    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    # Additional optimizer kwargs based on optimizer type
    opt_kwargs = {"lr": lr, "weight_decay": weight_decay}
    if optimizer_name == 'SGD':
        opt_kwargs['momentum'] = 0.9

    model = BlockRNNModel(
        model="GRU",
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        hidden_dim=hidden_size,
        n_rnn_layers=layer_size,
        n_epochs=30,
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=loss_functions[loss_name],
        optimizer_cls=optimizer_dict[optimizer_name],
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 5, "min_lr": 1e-7},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs=opt_kwargs,
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
        dataloader_kwargs={"num_workers": 11},
    )

    try:
        loaded_model = BlockRNNModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using the in-memory trained model.")
        loaded_model = model

    forecasts = loaded_model.historical_forecasts(
        val_scaled,
        forecast_horizon=output_chunk_length,
        stride=1,
        retrain=False,
        verbose=False
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "hidden_size": hidden_size,
        "layer_size": layer_size,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "loss_function": loss_name,
        "optimizer": optimizer_name,
        "overall_mae_val": overall_mae_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 10
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-25 17:11:10,290] A new study created in memory with name: no-name-11cecc3f-e79b-418d-be3e-4645e889bdee



Starting Trial 0
Hyperparameters: {'input_chunk_length': 34, 'output_chunk_length': 15, 'hidden_size': 43, 'layer_size': 3, 'dropout': 0.3016939481953512, 'batch_size': 116, 'lr': 6.840649584233789e-06, 'weight_decay': 0.008123911566977083, 'loss_fn': 'mae', 'optimizer': 'RMSprop'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.530
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.527
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.524
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 0.522
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.519
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.517
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.514
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.512
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.510
Metric val_loss improv

Model loaded from checkpoint for trial 0


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-25 17:12:07,036] Trial 0 finished with value: 0.4768506467382286 and parameters: {'input_chunk_length': 34, 'output_chunk_length': 15, 'hidden_size': 43, 'layer_size': 3, 'dropout': 0.3016939481953512, 'batch_size': 116, 'lr': 6.840649584233789e-06, 'weight_decay': 0.008123911566977083, 'loss_fn': 'mae', 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.4768506467382286.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCA

...

Current value: 0.19342538999316514, Current params: {'input_chunk_length': 31, 'output_chunk_length': 3, 'hidden_size': 198, 'layer_size': 1, 'dropout': 0.4483492338835919, 'batch_size': 102, 'lr': 0.0019973581728889075, 'weight_decay': 1.755040890111716e-06, 'loss_fn': 'mae', 'optimizer': 'AdamW'}
Best value: 0.08685611509641744, Best params: {'input_chunk_length': 24, 'output_chunk_length': 1, 'hidden_size': 188, 'layer_size': 1, 'dropout': 0.463770421977072, 'batch_size': 172, 'lr': 0.0020101615632169517, 'weight_decay': 4.080126913056516e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Starting Trial 68
Hyperparameters: {'input_chunk_length': 29, 'output_chunk_length': 6, 'hidden_size': 154, 'layer_size': 1, 'dropout': 0.414198293694559, 'batch_size': 249, 'lr': 0.0029019551263738566, 'weight_decay': 1.1758008993640984e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


[I 2025-05-25 18:06:05,262] Trial 68 pruned. Trial was pruned at epoch 0.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3497854497588795 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 0.04347294711306951, Current params: {'input_chunk_length': 29, 'output_chunk_length': 6, 'hidden_size': 154, 'layer_size': 1, 'dropout': 0.414198293694559, 'batch_size': 249, 'lr': 0.0029019551263738566, 'weight_decay': 1.1758008993640984e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}
Best value: 0.08685611509641744, Best params: {'input_chunk_length': 24, 'output_chunk_length': 1, 'hidden_size': 188, 'layer_size': 1, 'dropout': 0.463770421977072, 'batch_size': 172, 'lr': 0.0020101615632169517, 'weight_decay': 4.080126913056516e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Starting Trial 69
Hyperparameters: {'input_chunk_length': 25, 'output_chunk_length': 4, 'hidden_size': 177, 'layer_size': 1, 'dropout': 0.3497854497588795, 'batch_size': 169, 'lr': 0.001516449464241942, 'weight_decay': 1.3012251689743383e-05, 'loss_fn': 'mse', 'optimizer': 'SGD'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


[I 2025-05-25 18:06:06,966] Trial 69 pruned. Trial was pruned at epoch 0.


Current value: 0.3409546580825588, Current params: {'input_chunk_length': 25, 'output_chunk_length': 4, 'hidden_size': 177, 'layer_size': 1, 'dropout': 0.3497854497588795, 'batch_size': 169, 'lr': 0.001516449464241942, 'weight_decay': 1.3012251689743383e-05, 'loss_fn': 'mse', 'optimizer': 'SGD'}
Best value: 0.08685611509641744, Best params: {'input_chunk_length': 24, 'output_chunk_length': 1, 'hidden_size': 188, 'layer_size': 1, 'dropout': 0.463770421977072, 'batch_size': 172, 'lr': 0.0020101615632169517, 'weight_decay': 4.080126913056516e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}


In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.08685611509641744
Best Parameters: {'input_chunk_length': 24, 'output_chunk_length': 1, 'hidden_size': 188, 'layer_size': 1, 'dropout': 0.463770421977072, 'batch_size': 172, 'lr': 0.0020101615632169517, 'weight_decay': 4.080126913056516e-06, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
